## Finetuning

## Definition of the model 

### Original model

We start with the original model. There seems to be a competition how to define a network with the least possible amount of code. The slim library includes a **arg_scope** which allows define defaults for aguments.

In [81]:
tf.reset_default_graph()
with tf.variable_scope('vgg_16', [inputs]) as sc:
    with slim.arg_scope([slim.conv2d, slim.fully_connected],
                          activation_fn=tf.nn.relu,
                          weights_initializer=tf.truncated_normal_initializer(0.0, 0.01),
                          weights_regularizer=slim.l2_regularizer(0.0005)):
        images = tf.placeholder(tf.float32, [None, None, None, 3], name='images')
        inputs = tf.image.resize_images(images, (224,224))
        net = slim.repeat(inputs, 2, slim.conv2d, 64, [3, 3], scope='conv1')
        net = slim.max_pool2d(net, [2, 2], scope='pool1')
        net = slim.repeat(net, 2, slim.conv2d, 128, [3, 3], scope='conv2')
        net = slim.max_pool2d(net, [2, 2], scope='pool2')
        net = slim.repeat(net, 3, slim.conv2d, 256, [3, 3], scope='conv3')
        net = slim.max_pool2d(net, [2, 2], scope='pool3')
        net = slim.repeat(net, 3, slim.conv2d, 512, [3, 3], scope='conv4')
        net = slim.max_pool2d(net, [2, 2], scope='pool4')
        net = slim.repeat(net, 3, slim.conv2d, 512, [3, 3], scope='conv5')
        net = slim.max_pool2d(net, [2, 2], scope='pool5')
        net = slim.conv2d(net, 4096, [7, 7], padding='VALID', scope='fc6')
        net = slim.conv2d(net, 4096, [1, 1], scope='fc7')
        net = slim.conv2d(net, 1000, [1, 1], activation_fn=None, normalizer_fn=None,scope='fc8')

tf.train.SummaryWriter('/tmp/dumm/fine_tuning', tf.get_default_graph()).close()

In [82]:
variables_to_restore = slim.get_variables_to_restore()
init_assign_op, init_feed_dict = \
    slim.assign_from_checkpoint('/Users/oli/Dropbox/server_sync/tf_slim_models/vgg_16.ckpt', variables_to_restore)
sess = tf.Session()
sess.run(init_assign_op, init_feed_dict)

In [83]:
from imagenet_classes import class_names
img1 = imread('poodle.jpg')
feed_vals = [img1]
np.shape(feed_vals)
prob = sess.run(net, feed_dict={images:feed_vals})[0,0,0,]
preds = (np.argsort(prob)[::-1])[0:10]
for p in preds:
    print p, class_names[p], prob[p]

267 standard poodle 18.2776
265 toy poodle 12.6826
266 miniature poodle 12.4879
160 Afghan hound, Afghan 10.2724
221 Irish water spaniel 10.2163
355 llama 10.0407
244 Tibetan mastiff 9.9589
368 gibbon, Hylobates lar 9.12196
903 wig 9.04713
260 chow, chow chow 8.27523


## Finetuning

In [153]:
tf.reset_default_graph()
with tf.variable_scope('vgg_16', [inputs]) as sc:
    with slim.arg_scope([slim.conv2d, slim.fully_connected],
                          activation_fn=tf.nn.relu,
                          weights_initializer=tf.truncated_normal_initializer(0.0, 0.01),
                          weights_regularizer=slim.l2_regularizer(0.0005)):
        images = tf.placeholder(tf.float32, [None, None, None, 3], name='images')
        inputs = tf.image.resize_images(images, (224,224))
        net = slim.repeat(inputs, 2, slim.conv2d, 64, [3, 3], scope='conv1')
        net = slim.max_pool2d(net, [2, 2], scope='pool1')
        net = slim.repeat(net, 2, slim.conv2d, 128, [3, 3], scope='conv2')
        net = slim.max_pool2d(net, [2, 2], scope='pool2')
        net = slim.repeat(net, 3, slim.conv2d, 256, [3, 3], scope='conv3')
        net = slim.max_pool2d(net, [2, 2], scope='pool3')
        net = slim.repeat(net, 3, slim.conv2d, 512, [3, 3], scope='conv4')
        net = slim.max_pool2d(net, [2, 2], scope='pool4')
        net = slim.repeat(net, 3, slim.conv2d, 512, [3, 3], scope='conv5')
        net = slim.max_pool2d(net, [2, 2], scope='pool5')
        shape = int(np.prod(net.get_shape()[1:]))
        net = tf.reshape(net, [-1, shape])
        net = slim.fully_connected(net, 1000, scope='fc6') #Only 1000 
        net = slim.dropout(net, 0.5, scope='dropout6')
        net = slim.fully_connected(net, 10, scope='fc7', activation_fn=None, normalizer_fn=None) #Only 10
        # This would be the original vgg16. We replace these layers with FC layers and less classes
        #net = slim.conv2d(net, 4096, [7, 7], padding='VALID', scope='fc6')
        #net = slim.conv2d(net, 4096, [1, 1], scope='fc7')
        #net = slim.conv2d(net, 1000, [1, 1], activation_fn=None, normalizer_fn=None,scope='fc8')
       
tf.train.SummaryWriter('/tmp/dumm/fine_tuned', tf.get_default_graph()).close()

In [154]:
# Note we call the network studid, so that we don't get a hold for the tensors we need.
vars_to_restore = slim.get_variables(scope='vgg_16/conv1')
#vars_to_restore.append(slim.get_variables(scope='vgg_16/conv2'))
vars_to_restore

In [155]:
type(vars_to_restore)

list

In [156]:
sess = tf.Session()
sess.run(tf.initialize_all_variables())
restorer = tf.train.Saver(vars_to_restore)
restorer.restore(sess, '/Users/oli/Dropbox/server_sync/tf_slim_models/vgg_16.ckpt')
print("Model restored.")


Model restored.


In [159]:
sess.run(net, feed_dict={images:feed_vals})

array([[  1.26071405e-04,   4.15087328e-04,  -9.35655044e-05,
          7.45166559e-04,  -3.84996965e-04,  -1.29617998e-04,
         -3.65308631e-04,   2.90129654e-04,  -1.10490248e-04,
          7.67014455e-04]], dtype=float32)

### Learning TODO

In [160]:
sess.close()